# LLM

In [18]:
from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama3.2:1b", request_timeout=120.0)
resp = llm.complete("Who is Paul Graham?")
resp

CompletionResponse(text='Paul Graham is a renowned American computer scientist, entrepreneur, and programmer. He\'s best known for being the co-founder of Y Combinator, a seed investment firm that has backed many successful startups.\n\nGraham was born in 1961 in New York City. He studied at Harvard University and later earned his Ph.D. in theoretical physics from the University of California, Berkeley.\n\nBefore co-founding Y Combinator, Graham worked as a programmer for Apple and Microsoft. He also taught computer science at Stanford University and was an advisor to the US Department of Defense\'s Advanced Research Projects Agency (ARPA).\n\nIn 2007, Graham left his job at Y Combinator to pursue his passion for art and design. He started the Art20 organization, which focuses on supporting emerging artists and providing resources for creative projects.\n\nGraham is also known for his writing and public speaking skills. He\'s authored several books, including "The Hour Between Dog and 

In [19]:
resp.text

'Paul Graham is a renowned American computer scientist, entrepreneur, and programmer. He\'s best known for being the co-founder of Y Combinator, a seed investment firm that has backed many successful startups.\n\nGraham was born in 1961 in New York City. He studied at Harvard University and later earned his Ph.D. in theoretical physics from the University of California, Berkeley.\n\nBefore co-founding Y Combinator, Graham worked as a programmer for Apple and Microsoft. He also taught computer science at Stanford University and was an advisor to the US Department of Defense\'s Advanced Research Projects Agency (ARPA).\n\nIn 2007, Graham left his job at Y Combinator to pursue his passion for art and design. He started the Art20 organization, which focuses on supporting emerging artists and providing resources for creative projects.\n\nGraham is also known for his writing and public speaking skills. He\'s authored several books, including "The Hour Between Dog and Wolf" (a memoir about hi

## VectorDB

In [4]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama.llms import OllamaLLM

USER_AGENT environment variable not set, consider setting it to identify your requests.


# Testes

tgsc/sentence-transformer-ult5-pt-small
sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
BAAI/bge-small-en-v1.5

In [6]:
from langchain_openai import OpenAIEmbeddings

In [1]:
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import BeautifulSoupTransformer
from langchain_community.document_transformers import Html2TextTransformer
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# embeddings = HuggingFaceBgeEmbeddings(
#     model_name="BAAI/bge-small-en-v1.5",  # alternatively use "sentence-transformers/all-MiniLM-l6-v2" for a light and faster experience.
#     model_kwargs={'device':'cpu'}, 
#     encode_kwargs={'normalize_embeddings': True}
# )

loader = AsyncHtmlLoader(["https://hotmart.com/pt-br/blog/como-funciona-hotmart"])
docs = loader.load()
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)
docs_transformed[0].page_content[0:500]

USER_AGENT environment variable not set, consider setting it to identify your requests.
Fetching pages: 100%|##########| 1/1 [00:00<00:00,  7.63it/s]


'  * Primeiros passos \n  * Comece a vender \n  * Soluções da Hotmart \n  * Materiais gratuitos \n  * Categorias _ _\n\n    * Marketing Digital \n    * Empreendedorismo digital \n    * Produção de conteúdo \n    * Carreira no mundo digital \n    * Afiliado \n    * Produção de Vídeos \n    * Capacitação para creators \n    * Soluções da Hotmart \n    * Primeiros passos \n    * Comece a vender \n\n__\n\nBuscar\n\n_ _ pt-br\n\n  * Português - Brasil \n\nCadastre-se grátis\n\n  * Primeiros passos \n  * Comece a vender \n  * Solu'

In [2]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema.document import Document


def get_text_chunks_langchain(text):
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    docs = [Document(page_content=x) for x in text_splitter.split_text(text)]
    return docs


def main():
    text = "I want to use a langchain with a string instead of a txt file, is this possible?"
    docs = get_text_chunks_langchain(text)
    print(docs)


main()

[Document(metadata={}, page_content='I want to use a langchain with a string instead of a txt file, is this possible?')]


In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
import os

In [12]:
openai_api_key = os.getenv('OPENAI_API_KEY')
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs_transformed)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(openai_api_key=openai_api_key,model="text-embedding-3-small"),persist_directory="test/")

In [56]:
vectorstore = Chroma(embedding_function=OpenAIEmbeddings(), persist_directory="data/")

In [14]:
vectorstore.similarity_search("Como vender com a hotmart?",k=2)

[Document(metadata={'description': 'A Hotmart é uma plataforma grátis para você criar e vender cursos online, 100% fácil de usar e com ferramentas totalmente gratuitas', 'language': 'pt-br', 'source': 'https://hotmart.com/pt-br/blog/como-funciona-hotmart', 'title': 'O que é e como funciona a Hotmart?'}, page_content='Porém, o que oferecemos é muito mais do que um canal de hospedagem de conteúdo\ndigital e uma forma de aquisição. A Hotmart é um ecossistema completo e em\nconstante evolução para trazer ainda mais soluções para criar e escalar\nnegócios digitais.\n\nDados mostram que, na Hotmart, creators faturam 35% mais, sem mexer no esforço\noperacional.\n\nIsso graças ao rápido carregamento, alta taxa de aprovação, variedade de\nformas de pagamento, usabilidade agradável e ferramentas específicas para\naumentar as vendas, como o Order Bump e Funil de Vendas, por exemplo.\n\n**VÍDEO: Como funciona a Hotmart? Tudo que você precisa saber para vender na\nHotmart!  \n\n**\n\n### **Quanto a

In [32]:
retriever = vectorstore.as_retriever(k=2)
prompt = hub.pull("rlm/rag-prompt")

llm = OllamaLLM(model="llama3.2:1b")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



/home/felipelangoniramos/Documents/desafio_hm/hot_env/lib/python3.10/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [37]:
prompt.format(context=vectorstore.similarity_search("Como vender com a hotmart?",k=2)[0].page_content, question="Como vender com a hotmart?")

"Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: Como vender com a hotmart? \nContext: ## Como vender com a Hotmart?\n\nO primeiro passo para vender com a Hotmart é fazer o cadastro gratuito.\n\nSe você quer criar seus produtos ou comunidade, o passo seguinte é justamente\nfazer essa configuração na plataforma.\n\nPara quem busca um negócio online a partir da afiliação, depois do cadastro,\nvocê deve passear pelo mercado de afiliação e conhecer melhor as suas opções.\n\nO blog está recheado de conteúdos para te ajudar nessas etapas.\n\nFeito isso, as vendas são com você: aposte em estratégias de marketing\ndigital, construa uma audiência e divulgue os seus produtos como achar melhor,\nsempre fazendo ajustes para vender mais.\n\nVocê pode monitorar tudo isso em sua página dentro da 

In [38]:
def run(input):
    return llm.invoke(prompt.format(context=vectorstore.similarity_search(input,k=2)[0].page_content, question=input))

In [39]:
run("Como vender com a hotmart?")

'Para vender com a Hotmart, você precisa seguir os passos abaixo:\n\n1. Fazer o cadastro gratuito na plataforma.\n2. Configurar seus produtos ou comunidade no site.\n3. Monitorear todas as etapas em sua página para ajustar estratégias de marketing digital e aumentar vendas.\n\nSe tiver alguma dúvida sobre como funcionam essas etapas, sugiro procurar mais informações na plataforma.'

In [33]:
rag_chain.invoke("Como vender com a hotmart?")

'A Hotmart é uma plataforma para venda de conteúdos digitais completa que oferece estratégias de marketing digital para criadores de conteúdo, produtores e afiliados. A plataforma permite que os usuários vendem produtos online e recebe comissões por cada venda realizada. Os produtores podem criar seus produtos ou comunidade na plataforma e os afiliados podem escolher entre centenas de produtos para promover e receber comissões por cada venda.'

# Chunking Evaluation

In [15]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.evaluation import (
    DatasetGenerator,
    FaithfulnessEvaluator,
    RelevancyEvaluator
)


import time

ModuleNotFoundError: No module named 'llama_index.evaluation'

In [14]:
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import BeautifulSoupTransformer

loader = AsyncHtmlLoader(["https://hotmart.com/pt-br/blog/como-funciona-hotmart"])
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.
Fetching pages: 100%|##########| 1/1 [00:01<00:00,  1.19s/it]


In [16]:
from langchain_community.document_transformers import Html2TextTransformer

html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)
docs_transformed[0].page_content[0:500]

'  * Primeiros passos \n  * Comece a vender \n  * Soluções da Hotmart \n  * Materiais gratuitos \n  * Categorias _ _\n\n    * Marketing Digital \n    * Empreendedorismo digital \n    * Produção de conteúdo \n    * Carreira no mundo digital \n    * Afiliado \n    * Produção de Vídeos \n    * Capacitação para creators \n    * Soluções da Hotmart \n    * Primeiros passos \n    * Comece a vender \n\n__\n\nBuscar\n\n_ _ pt-br\n\n  * Português - Brasil \n\nCadastre-se grátis\n\n  * Primeiros passos \n  * Comece a vender \n  * Solu'

In [17]:
docs_transformed

[Document(metadata={'source': 'https://hotmart.com/pt-br/blog/como-funciona-hotmart', 'title': 'O que é e como funciona a Hotmart?', 'description': 'A Hotmart é uma plataforma grátis para você criar e vender cursos online, 100% fácil de usar e com ferramentas totalmente gratuitas', 'language': 'pt-br'}, page_content='  * Primeiros passos \n  * Comece a vender \n  * Soluções da Hotmart \n  * Materiais gratuitos \n  * Categorias _ _\n\n    * Marketing Digital \n    * Empreendedorismo digital \n    * Produção de conteúdo \n    * Carreira no mundo digital \n    * Afiliado \n    * Produção de Vídeos \n    * Capacitação para creators \n    * Soluções da Hotmart \n    * Primeiros passos \n    * Comece a vender \n\n__\n\nBuscar\n\n_ _ pt-br\n\n  * Português - Brasil \n\nCadastre-se grátis\n\n  * Primeiros passos \n  * Comece a vender \n  * Soluções da Hotmart \n  * Materiais gratuitos \n  * Categorias _ _\n\n    * Marketing Digital \n    * Empreendedorismo digital \n    * Produção de conteúdo 

In [ ]:
documents = SimpleDirectoryReader("./data/10k/").load_data()

# 